In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from datetime import datetime
import os
from glob import glob #para eliminar archivos dentro de carpeta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
import itertools
from plotly.subplots import make_subplots

In [2]:
def inicio_driver(link:str):
    service = Service(ChromeDriverManager().install())
    carpeta_descarga=os.getcwd().replace('src','downloads')
    prefs = {'download.default_directory' : carpeta_descarga,
        "directory_upgrade": True}
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(link)
    driver.maximize_window()
    return driver


def every_downloads_chrome(driver):
    '''Para ver cuando terminan las descargas'''
    if not driver.current_url.startswith("chrome://downloads"):
        driver.get("chrome://downloads/")
    return driver.execute_script("""
        var items = document.querySelector('downloads-manager')
            .shadowRoot.getElementById('downloadsList').items;
        if (items.every(e => e.state === "COMPLETE"))
            return items.map(e => e.fileUrl || e.file_url);
        """)
    
def scrap_usos_trim():
    link_usos_trim = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_indicesimpue_04.xls"
    driver = inicio_driver(link_usos_trim)
    # waits for all the files to be completed and returns the paths
    paths = WebDriverWait(driver, 120, 1).until(every_downloads_chrome)
    

In [3]:
def create_repeated_years(df:pd.DataFrame):
    start_year = 1986
    end_year = start_year + int(np.ceil(149 / 4)) - 1
    years = range(start_year, end_year + 1)
    years_repeated = np.repeat(list(years), 4)[:149]
    df['año'] = years_repeated
    return df

def create_repeated_trim(df:pd.DataFrame):
    values = ["I", "II", "III", "IV"]

    # Utilizar la función cycle para repetir los valores indefinidamente
    values_repeated = itertools.cycle(values)

    # Crear una nueva columna en el dataframe con los valores repetidos
    df["trimestre"] = [next(values_repeated) for _ in range(len(df))]
    df["período"] = df["trimestre"].astype(str) + "-" + df["año"].astype(str)
    return df

def wrangling_usos_trim():
    header = ["período","ng_v","ng_p","ng_q","bk_v","bk_p","bk_q","bi_v","bi_p","bi_q","cyl_v","cyl_p","cyl_q","abk_v","abk_p","abk_q","bc_v","bc_p","bc_q","vya_v","vya_p","vya_q"]
    df = pd.read_excel("../downloads/sh_indicesimpue_04.xls", header=4,skipfooter=6)
    df = df.dropna(axis=1)
    df.columns = header
    df = create_repeated_years(df)
    df = create_repeated_trim(df)
    df = df.set_index("período")
    df = df.drop(["año","trimestre"],axis=1)
    return df

def wrangling_usos_trim_var():
    df = wrangling_usos_trim()
    df = df.pct_change(4)
    return df.dropna()

   

In [4]:
scrap_usos_trim()

In [5]:
def subplot_var_indice(anio_desde:str="2015"):
    anio_desde = str(anio_desde)
    # Crear el subplot con 2 columnas y 4 filas
    df = wrangling_usos_trim_var()[f"I-{anio_desde}":]
    q_columns = wrangling_usos_trim_var().columns[wrangling_usos_trim_var().columns.str.endswith("q")].drop("ng_q")
    fig = make_subplots(rows=3, cols=2, subplot_titles=["Bienes de capital","Bienes intermedios", "combustibles y lubricantes", "Acc. para bienes de capital", "Bienes de consumo", "Vehículos y automotores"])

    # Crear un bucle para añadir cada subplot
    for i, col in enumerate(q_columns, start=1):
        row = (i-1) // 2 + 1
        column = (i-1) % 2 + 1
        
        # Añadir el trace correspondiente al subplot
        trace = go.Scatter(x=df.index, y=df[col], name=col)
        fig.add_trace(trace, row=row, col=column)

    # Actualizar las propiedades del gráfico
    fig.update_layout(height=900, width=800, title='Evolución de las cantidades importadas según usos económicos<br>Variación interanual', template = None,
                    font_family = "verdana", showlegend = False)
    fig.update_yaxes(tickformat = ".0%")
    note = 'Fuente: INDEC'
    fig.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper',x=0.1,yref='paper', y=-0.11, 
                            xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    fig.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
        xref='paper', yref='paper', x=0.5, y=0.5)
    # Mostrar el gráfico
    return fig

subplot_var_indice(anio_desde=2012)


In [6]:
def subplot_indice(anio_desde:str = 2015):
    anio_desde = str(anio_desde)
# Crear el subplot con 2 columnas y 4 filas
    df = wrangling_usos_trim()[f"I-{anio_desde}":]
    q_columns = wrangling_usos_trim().columns[wrangling_usos_trim().columns.str.endswith("q")].drop("ng_q")
    fig = make_subplots(rows=3, cols=2, subplot_titles=["Bienes de capital","Bienes intermedios", "combustibles y lubricantes", "Acc. para bienes de capital", "Bienes de consumo", "Vehículos y automotores"])

    # Crear un bucle para añadir cada subplot
    for i, col in enumerate(q_columns, start=1):
        row = (i-1) // 2 + 1
        column = (i-1) % 2 + 1
        
        # Añadir el trace correspondiente al subplot
        trace = go.Scatter(x=df.index, y=df[col], name=col)
        fig.add_trace(trace, row=row, col=column)
        last_value = df[col].mean()
        fig.add_shape(
            type="line",
            x0=df.index[0],
            x1=df.index[-1],
            y0=last_value,
            y1=last_value,
            line=dict(color=trace.line.color, dash="dash"),
            row=row,
            col=column,
        )

    # Actualizar las propiedades del gráfico
    fig.update_layout(height=900, width=800, title='Evolución de las cantidades importadas según usos económicos<br>índice 2004 = 100', template = None,
                    font_family = "verdana", showlegend = False)
    # fig.update_yaxes(tickformat = ".0%")
    note = f'Fuente: INDEC. La línea punteada representa la media de {anio_desde} en adelante'
    fig.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper',x=0.7,yref='paper', y=-0.11, 
                            xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    fig.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
        xref='paper', yref='paper', x=0.5, y=0.5)

    return fig

subplot_indice(anio_desde=2012)

In [7]:
# for i in glob("../downloads/*", recursive = True): os.remove(i)